In [ ]:
import pathlib

import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [ ]:
print(tf.__version__)

In [ ]:
dataset_path = keras.utils.get_file("auto-mpg.data", "http://archive.ics.uci.edu/ml/machine-learning-databases/auto-mpg/auto-mpg.data")
dataset_path

In [ ]:
column_names = ['MPG', 'Cylinders','Displacement', 'Horsepower', 'Weight', 'Acceleration', 'Model Year', 'Origin']

raw_data = pd.read_csv(dataset_path, names = column_names, na_values= "?", comment='\t', sep = " ", skipinitialspace=True)

In [ ]:
dataset = raw_data.copy()

dataset.tail()

In [ ]:
dataset.isna().sum()

In [ ]:
dataset = dataset.dropna()

In [ ]:
origin = dataset.pop('Origin')

In [ ]:
dataset['USA'] = (origin == 1) * 1.0
dataset['Europe'] = (origin == 2) * 1.0
dataset['Japen'] = (origin == 3) * 1.0

dataset.tail()

In [ ]:
train_dataset = dataset.sample(frac = 0.8, random_state = 42)
test_dataset = dataset.drop(train_dataset.index)

In [ ]:
train_dataset

In [ ]:
test_dataset.shape

In [ ]:
sns.pairplot(train_dataset[['MPG', 'Cylinders','Displacement', 'Horsepower', 'Weight', 'Acceleration', 'Model Year']], diag_kind= "kde")

In [ ]:
train_stats = train_dataset.describe()
train_stats.pop('MPG')

train_stats = train_stats.transpose()
train_stats

In [ ]:
train_labels = train_dataset.pop('MPG')
test_labels = test_dataset.pop('MPG')

In [ ]:
def norm(x):
    return (x - train_stats['mean']) / train_stats['std']

In [ ]:
normed_train_data = norm(train_dataset)
normed_test_data = norm(test_dataset)

In [ ]:
def build_model():
    model = keras.Sequential([
        layers.Dense(64, activation='relu', input_shape=[len(train_dataset.keys())]),
        layers.Dense(64, activation='relu'),
        layers.Dense(1),
    ])

    optimizer = keras.optimizers.RMSprop(0.001)

    model.compile(optimizer = optimizer, loss = 'mse', metrics = ['mae', 'mse'])

    return model

In [ ]:
model = build_model()

In [ ]:
model.summary()

In [ ]:
example_batch = normed_train_data[:10]

example_results = model.predict(example_batch)
example_results

In [ ]:
class PrintDot(keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs):
        if epoch % 100 == 0:
            print('')
        print('.', end='')

In [ ]:
EPOCHS = 1000

history = model.fit(normed_train_data, train_labels, epochs=EPOCHS, validation_split=0.2, verbose=0, callbacks=[PrintDot()])

In [ ]:
hist = pd.DataFrame(history.history)

hist['epoch'] = history.epoch

hist.tail()

In [ ]:
def plot_history(history):
    hist = pd.DataFrame(history.history)
    hist['epoch'] = history.epoch

    plt.figure()
    plt.xlabel('Epoch')
    plt.ylabel('Mean Abs Error [MPG]')

    plt.plot(hist['epoch'], hist['mae'], label='Train error')
    plt.plot(hist['epoch'], hist['val_mae'], label='Val error')

    plt.ylim([0,5])
    plt.legend()

    plt.figure()
    plt.xlabel('Epoch')
    plt.ylabel('Mean Square Error [$MPG^2$]')

    plt.plot(hist['epoch'], hist['mse'], label='Train error')
    plt.plot(hist['epoch'], hist['val_mse'], label='Val error')

    plt.ylim([0, 20])
    plt.legend()

    plt.show()

In [ ]:
plot_history(history)

In [ ]:
model = build_model()

early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)

history = model.fit(normed_train_data, train_labels, epochs = EPOCHS, validation_split=0.2, verbose=0, callbacks=[early_stop, PrintDot()])

In [ ]:
plot_history(history)

In [ ]:
loss, mae, mse = model.evaluate(normed_test_data, test_labels, verbose=2)

print("Testing set mean Abs error: {:5.2f} MPG".format(mae))

In [ ]:
test_predictions = model.predict(normed_test_data).flatten()

In [ ]:
plt.scatter(test_labels, test_predictions)

plt.xlabel('True values [MPG]')
plt.ylabel('Predictions [MPG]')

plt.axis('equal')
plt.axis('square')

plt.xlim([0, plt.xlim()[1]])
plt.ylim([0, plt.ylim()[1]])

_ = plt.plot([-100, 100], [-100, 100])

In [ ]:
error = test_predictions - test_labels

plt.hist(error, bins = 100)
plt.xlabel('Prediction Error [MPG]')
_ = plt.ylabel('Count')